In [ ]:
import numpy as np
import pandas as pd
import vectorbt as vbt

# =============================
# PARAMÈTRES
# =============================
rsi_length = 14
lookback_left = 3
lookback_right = 3

min_div_bars = 10
max_div_bars = 200

use_close_confirmation = True
use_volume_confirmation = False
use_trend_filter = True   # 🔥 maintenant OK

ema_length = 200

stop_loss_pct = 3.0
take_profit_pct = 6.0
trailing_stop = True

use_hidden_div = True
use_regular_div = True

rsi_zone = "Both"  # "Both", "Oversold Only", "Overbought Only"

# =============================
# DONNÉES
# =============================
df = data.copy()
df.columns = df.columns.str.lower()

close = df['close']
high = df['high']
low = df['low']
volume = df['volume']

# =============================
# INDICATEURS
# =============================
rsi = vbt.RSI.run(close, window=rsi_length).rsi
ema = vbt.MA.run(close, window=ema_length).ma
ema_shifted = ema.shift(1)   # 🔥 CORRECTION CRITIQUE

volume_avg = volume.rolling(20).mean()
volume_high = volume > volume_avg * 1.3

entry_delay = 1 if use_close_confirmation else 0

# =============================
# SIGNALS
# =============================
long_entries = np.zeros(len(df), dtype=bool)
short_entries = np.zeros(len(df), dtype=bool)

price_highs, rsi_highs, bar_highs = [], [], []
price_lows, rsi_lows, bar_lows = [], [], []

# =============================
# BOUCLE PRINCIPALE
# =============================
for i in range(len(df)):

    # ========= PIVOTS =========
    if i >= lookback_left and i + lookback_right < len(df):

        if high[i] == high[i-lookback_left:i+lookback_right+1].max():
            price_highs.insert(0, high[i])
            rsi_highs.insert(0, rsi[i])
            bar_highs.insert(0, i)

        if low[i] == low[i-lookback_left:i+lookback_right+1].min():
            price_lows.insert(0, low[i])
            rsi_lows.insert(0, rsi[i])
            bar_lows.insert(0, i)

        price_highs = price_highs[:10]
        rsi_highs = rsi_highs[:10]
        bar_highs = bar_highs[:10]

        price_lows = price_lows[:10]
        rsi_lows = rsi_lows[:10]
        bar_lows = bar_lows[:10]

    bullish = bearish = False
    hidden_bullish = hidden_bearish = False
    signal_bar = None

    # ===== REGULAR =====
    if use_regular_div and len(price_highs) >= 2:
        bars = abs(bar_highs[0] - bar_highs[1])
        if price_highs[0] > price_highs[1] and rsi_highs[0] < rsi_highs[1] and min_div_bars <= bars <= max_div_bars:
            bearish = True
            signal_bar = bar_highs[0]

    if use_regular_div and len(price_lows) >= 2:
        bars = abs(bar_lows[0] - bar_lows[1])
        if price_lows[0] < price_lows[1] and rsi_lows[0] > rsi_lows[1] and min_div_bars <= bars <= max_div_bars:
            bullish = True
            signal_bar = bar_lows[0]

    # ===== HIDDEN =====
    if use_hidden_div and len(price_highs) >= 2:
        bars = abs(bar_highs[0] - bar_highs[1])
        if price_highs[0] < price_highs[1] and rsi_highs[0] > rsi_highs[1] and min_div_bars <= bars <= max_div_bars:
            hidden_bearish = True
            signal_bar = bar_highs[0]

    if use_hidden_div and len(price_lows) >= 2:
        bars = abs(bar_lows[0] - bar_lows[1])
        if price_lows[0] > price_lows[1] and rsi_lows[0] < rsi_lows[1] and min_div_bars <= bars <= max_div_bars:
            hidden_bullish = True
            signal_bar = bar_lows[0]

    # ===== CONDITIONS BRUTES =====
    long_cond = False
    short_cond = False

    if bullish and rsi_zone in ["Both", "Oversold Only"]:
        long_cond = True

    if hidden_bullish and rsi_zone in ["Both", "Overbought Only"]:
        long_cond = True

    if bearish and rsi_zone in ["Both", "Overbought Only"]:
        short_cond = True

    if hidden_bearish and rsi_zone in ["Both", "Oversold Only"]:
        short_cond = True

    # ===== ENTRÉE EXACTE PINE =====
    if signal_bar is not None and i == signal_bar + entry_delay:

        if use_trend_filter:
            long_cond &= close[i] > ema_shifted[i]
            short_cond &= close[i] < ema_shifted[i]

        if use_volume_confirmation:
            long_cond &= volume_high[i]
            short_cond &= volume_high[i]

        long_entries[i] = long_cond
        short_entries[i] = short_cond
